In [1]:
%cd ~/bioinfo1/project/

/rna/hyemin/bioinfo1/project


In [1]:
!fastq-dump --gzip SRR458758
!fastq-dump --gzip SRR458759
!fastq-dump --gzip SRR458760

^C


In [6]:
!fastp -i CLIP/rawdata/SRR458758.fastq.gz -q 25 -l 20 -a CTGTAGGCACCATCAAT \
    -j CLIP/reports/CLIP-35L33G-process.json -h CLIP/reports/CLIP-35L33G-process.html -w 10 -o CLIP/processed/CLIP-35L33G.filtered.fq.gz
!fastp -i CLIP/rawdata/SRR458759.fastq.gz -q 25 -l 20 -a CTGTAGGCACCATCAAT \
    -j CLIP/reports/CLIP-2J3-process.json -h CLIP/reports/CLIP-2J3-process.html -w 10 -o CLIP/processed/CLIP-2J3.filtered.fq.gz
!fastp -i CLIP/rawdata/SRR458760.fastq.gz -q 25 -l 20 -a CTGTAGGCACCATCAAT \
    -j CLIP/reports/CLIP-Polyclonal-process.json -h CLIP/reports/CLIP-Polyclonal-process.html -w 10 -o CLIP/processed/CLIP-Polyclonal.filtered.fq.gz
!fastp -i CLIP/rawdata/SRR458753.fastq.gz -q 25 -l 20 -a CTGTAGGCACCATCAAT \
    -j CLIP/reports/RNA-control-process.json -h CLIP/reports/RNA-control-process.html -w 10 -o CLIP/processed/RNA-control.filtered.fq.gz

Read1 before filtering:
total reads: 33548802
total bases: 2616806556
Q20 bases: 2334372244(89.2069%)
Q30 bases: 2126309601(81.2559%)

Read1 after filtering:
total reads: 26626307
total bases: 1130387931
Q20 bases: 1097341146(97.0765%)
Q30 bases: 1009410106(89.2977%)

Filtering result:
reads passed filter: 26626307
reads failed due to low quality: 1245020
reads failed due to too many N: 10088
reads failed due to too short: 5667387
reads with adapter trimmed: 29718589
bases trimmed due to adapters: 1340326961

Duplication rate (may be overestimated since this is SE data): 54.0198%

JSON report: reports/CLIP-2J3-process.json
HTML report: reports/CLIP-2J3-process.html

fastp -i rawdata/SRR458759.fastq.gz -q 25 -l 20 -a CTGTAGGCACCATCAAT -j reports/CLIP-2J3-process.json -h reports/CLIP-2J3-process.html -w 10 -o processed/CLIP-2J3.filtered.fq.gz 
fastp v0.23.4, time used: 50 seconds
Read1 before filtering:
total reads: 30117545
total bases: 2349168510
Q20 bases: 2030835960(86.4491%)
Q30 bas

In [17]:
Fa = 'binfo1-datapack1/GRCm39.primary_assembly.genome.fa'
with open(Fa, 'r') as fIn, open(Fa[:-len('.fa')] + '.basic.fa', 'w') as fOut:
    lines = fIn.readlines()
    for line in lines:
        if line.startswith('>chr'):
            fOut.write(line.split(' ')[0] + '\n')
        elif line.startswith('>'):
            break
        else:
            fOut.write(line)

#### Alignment with gsnap

In [2]:
!/rna/modules/local/gmap-2016-11-07/bin/gmap_build -D CLIP/reference/ -d GRCm39.basic.index -k 12 binfo1-datapack1/GRCm39.primary_assembly.genome.basic.fa

Sorting chromosomes in chrom order.  To turn off or sort other ways, use the -s flag.
Creating files in directory CLIP/reference//GRCm39.basic.index
Running "/rna/modules/local/gmap-2016-11-07/bin/fa_coords"     -o "CLIP/reference//GRCm39.basic.index.coords" -f "CLIP/reference//GRCm39.basic.index.sources"
Opening file binfo1-datapack1/GRCm39.primary_assembly.genome.basic.fa
  Contig chr1: concatenated at chromosome end: chr1:1..195154279 (length = 195154279 nt)
  Contig chr2: concatenated at chromosome end: chr2:1..181755017 (length = 181755017 nt)
  Contig chr3: concatenated at chromosome end: chr3:1..159745316 (length = 159745316 nt)
  Contig chr4: concatenated at chromosome end: chr4:1..156860686 (length = 156860686 nt)
  Contig chr5: concatenated at chromosome end: chr5:1..151758149 (length = 151758149 nt)
  Contig chr6: concatenated at chromosome end: chr6:1..149588044 (length = 149588044 nt)
  Contig chr7: concatenated at chromosome end: chr7:1..144995196 (length = 144995196 nt)


In [ ]:
!cat binfo1-datapack1/gencode.gtf | gtf_splicesites > CLIP/reference/gencode.splicesites
!cat binfo1-datapack1/gencode.gtf | gtf_introns > CLIP/reference/gencode.introns

In [ ]:
!cat CLIP/reference/gencode.splicesites | iit_store -o CLIP/reference/gencode.splicesites.iit
!cat CLIP/reference/gencode.introns | iit_store -o CLIP/reference/gencode.introns.iit

In [ ]:
!zcat CLIP/processed/CLIP-35L33G.filtered.fq.gz | \
    nice -n 10 gsnap -D CLIP/reference/ -d GRCm39.basic.index -m 0.1 -k 12 -N 1 -s CLIP/reference/gencode.splicesites.iit -t 128 -A sam -o CLIP/alignments/CLIP-35L33G.sam
!samtools view -Sb CLIP/alignments/CLIP-35L33G.sam | samtools sort -@ 10 -o CLIP/alignments/CLIP-35L33G.bam
!samtools index CLIP/alignments/CLIP-35L33G.bam

In [10]:
!zcat CLIP/processed/CLIP-2J3.filtered.fq.gz | \
    nice -n 10 gsnap -D CLIP/reference/ -d GRCm39.basic.index -m 0.1 -k 12 -N 1 -s CLIP/reference/gencode.splicesites.iit -t 128 -A sam -o CLIP/alignments/CLIP-2J3.sam
!samtools view -Sb CLIP/alignments/CLIP-2J3.sam | samtools sort -@ 10 -o CLIP/alignments/CLIP-2J3.bam
!samtools index CLIP/alignments/CLIP-2J3.bam

zsh:1: command not found: gsnap
[main_samview] fail to read the header from "-".
samtools sort: failed to read header from "-"
[E::hts_open_format] Failed to open file "CLIP/alignments/CLIP-2J3.bam" : No such file or directory
samtools index: failed to open "CLIP/alignments/CLIP-2J3.bam": No such file or directory


In [ ]:
!zcat CLIP/processed/CLIP-Polyclonal.filtered.fq.gz | \
    nice -n 10 gsnap -D CLIP/reference/ -d GRCm39.basic.index -m 0.1 -k 12 -N 1 -s CLIP/reference/gencode.splicesites.iit -t 128 -A sam -o CLIP/alignments/CLIP-Polyclonal.sam
!samtools view -Sb CLIP/alignments/CLIP-Polyclonal.sam | samtools sort -@ 10 -o CLIP/alignments/CLIP-Polyclonal.bam
!samtools index CLIP/alignments/CLIP-Polyclonal.bam

In [ ]:
!zcat CLIP/processed/RNA-control.filtered.fq.gz | \
    nice -n 10 gsnap -D CLIP/reference/ -d GRCm39.basic.index -m 0.1 -k 12 -N 1 -s CLIP/reference/gencode.splicesites.iit -t 128 -A sam -o CLIP/alignments/RNA-control.sam
!samtools view -Sb CLIP/alignments/RNA-control.sam | samtools sort -@ 10 -o CLIP/alignments/RNA-control.bam
!samtools index CLIP/alignments/RNA-control.bam

In [ ]:
!samtools view -F 3844 -b CLIP/alignments/CLIP-35L33G.bam > CLIP/alignments/CLIP-35L33G.filtered.bam
!samtools view -F 3844 -b CLIP/alignments/CLIP-2J3.bam > CLIP/alignments/CLIP-2J3.filtered.bam
!samtools view -F 3844 -b CLIP/alignments/CLIP-Polyclonal.bam > CLIP/alignments/CLIP-Polyclonal.filtered.bam
!samtools view -F 3844 -b CLIP/alignments/RNA-control.bam > CLIP/alignments/RNA-control.filtered.bam
# mapped, primary alignment, quality checked, no duplicate or supplementary reads

#### Alignment with STAR

In [ ]:
!nice -n 10 STAR --runThreadN 128 --runMode genomeGenerate --genomeDir CLIP/reference/star_index_78nt \
    --genomeFastaFiles binfo1-datapack1/GRCm39.primary_assembly.genome.basic.fa --sjdbGTFfile binfo1-datapack1/gencode.gtf --sjdbOverhang 77

In [ ]:
!nice -n 10 STAR --runThreadN 128 --runMode genomeGenerate --genomeDir CLIP/reference/star_index_54nt \
    --genomeFastaFiles binfo1-datapack1/GRCm39.primary_assembly.genome.basic.fa --sjdbGTFfile binfo1-datapack1/gencode.gtf --sjdbOverhang 53

In [ ]:
!nice -n 10 STAR --runThreadN 128 --genomeDir CLIP/reference/star_index_78nt --readFilesIn CLIP/processed/CLIP-35L33G.filtered.fq.gz --readFilesCommand zcat \
    --outFilterMismatchNoverLmax 0.1 --outSAMprimaryFlag AllBestScore --outSAMtype BAM Unsorted --outFileNamePrefix CLIP/alignments/star/CLIP-35L33G
!nice -n 10 STAR --runThreadN 128 --genomeDir CLIP/reference/star_index_78nt --readFilesIn CLIP/processed/CLIP-2J3.filtered.fq.gz --readFilesCommand zcat \
    --outFilterMismatchNoverLmax 0.1 --outSAMprimaryFlag AllBestScore --outSAMtype BAM Unsorted --outFileNamePrefix CLIP/alignments/star/CLIP-2J3
!nice -n 10 STAR --runThreadN 128 --genomeDir CLIP/reference/star_index_78nt --readFilesIn CLIP/processed/CLIP-Polyclonal.filtered.fq.gz --readFilesCommand zcat \
    --outFilterMismatchNoverLmax 0.1 --outSAMprimaryFlag AllBestScore --outSAMtype BAM Unsorted --outFileNamePrefix CLIP/alignments/star/CLIP-Polyclonal

In [ ]:
!nice -n 10 STAR --runThreadN 128 --genomeDir CLIP/reference/star_index_54nt --readFilesIn CLIP/processed/RNA-control.filtered.fq.gz --readFilesCommand zcat \
    --outFilterMismatchNoverLmax 0.1 --outSAMprimaryFlag AllBestScore --outSAMtype BAM Unsorted --outFileNamePrefix CLIP/alignments/star/RNA-control

In [ ]:
!samtools sort -o CLIP/alignments/star/CLIP-35L33G.bam -@ 20 CLIP/alignments/star/CLIP-35L33GAligned.out.bam
!samtools sort -o CLIP/alignments/star/CLIP-2J3.bam -@ 20 CLIP/alignments/star/CLIP-2J3Aligned.out.bam
!samtools sort -o CLIP/alignments/star/CLIP-Polyclonal.bam -@ 20 CLIP/alignments/star/CLIP-PolyclonalAligned.out.bam
!samtools sort -o CLIP/alignments/star/RNA-control.bam -@ 20 CLIP/alignments/star/RNA-controlAligned.out.bam

In [ ]:
!samtools index CLIP/alignments/star/CLIP-35L33G.bam
!samtools index CLIP/alignments/star/CLIP-2J3.bam
!samtools index CLIP/alignments/star/CLIP-Polyclonal.bam
!samtools index CLIP/alignments/star/RNA-control.bam

In [2]:
!samtools view -F 3844 -b CLIP/alignments/star/CLIP-35L33G.bam > CLIP/alignments/star/CLIP-35L33G.filtered.bam
!samtools view -F 3844 -b CLIP/alignments/star/CLIP-2J3.bam > CLIP/alignments/star/CLIP-2J3.filtered.bam
!samtools view -F 3844 -b CLIP/alignments/star/CLIP-Polyclonal.bam > CLIP/alignments/star/CLIP-Polyclonal.filtered.bam
!samtools view -F 3844 -b CLIP/alignments/star/RNA-control.bam > CLIP/alignments/star/RNA-control.filtered.bam
# mapped, primary alignment, quality checked, no duplicate or supplementary reads